In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('data/kendrick_lamar.csv')

In [4]:
print(data.head())
print(data['Lyrics'].head())

                Track Title Release Title Release Subtitle  Track #  \
0       Fuck Your Ethnicity    Section.80              NaN        1   
1                   Hol' Up    Section.80              NaN        2   
2                  A.D.H.D.    Section.80              NaN        3   
3     No Make-Up (Her Vice)    Section.80              NaN        4   
4  Tammy's Song (Her Evils)    Section.80              NaN        5   

  Track Length (mm:ss)     Featuring Release Type  Release Type #  \
0                 3:44  Colin Munroe       Studio             1.0   
1                 2:53           NaN       Studio             1.0   
2                 3:35           NaN       Studio             1.0   
3                 3:55           NaN       Studio             1.0   
4                 2:41           NaN       Studio             1.0   

  Release Date  # of Tracks  ... Jaguar Maserati McLaren Mercedes-Benz  \
0     2 Jul 11         16.0  ...    NaN      NaN     NaN           1.0   
1     2 Ju

In [5]:
print(data['Lyrics'][0])

Gather 'round
 I'm glad everybody came out tonight
 As we stand on our neighborhood corner
 Know that this fire that's burning represents the passion you have
 Listen
 Keisha, Tammy, come up front
 I recognize all of you
 Every creed and color
 With that being said
 Fuck your ethnicity
 You understand that?
 We gon' talk about a lot of shit that concerns you, all of you
 Now everybody, throw your hands up high
 If you don't give a fuck, throw your hands up high
 Throw your hands up high
 Now, I don't give a fuck if you
 Black, white, Asian, Hispanic, goddammit
 That don't mean shit to me
 Fuck your ethnicity, nigga
 Fire burning inside my eyes, this the music that saved my life
 Y'all be calling it hip-hop, I be calling it hypnotize
 Yeah, hypnotize, trapped my body, but freed my mind
 What the fuck are you fighting for? Ain't nobody gon' win that war
 My details be retail, man, I got so much in store
 Racism is still alive, yellow tape and colored lines
 Fuck that, nigga, look at that

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vishruth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
stop_words = stopwords.words('english')
stop_words.extend(['nigga', 'niggas', 'shit', 'fuck', 'fucking',
                  'fucked', 'bitch', 'fuckin', 'go', 'got', 'gotta', 'oh', 'get', 'huh', 'yeah' ,'uh', 'man'])
vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=0.1)
tfidf = vectorizer.fit_transform(data['Lyrics'].values.astype('U'))
print("Term-Document matrix shape: ", tfidf.A.shape)
print("No. of words: ", len(vectorizer.get_feature_names_out()))


Term-Document matrix shape:  (237, 344)
No. of words:  344


In [9]:
from sklearn.decomposition import NMF

In [10]:
nmf = NMF(n_components=5, init="nndsvda")
W = nmf.fit_transform(tfidf)
print("Topic weight for each lyric - matrix shape: ", W.shape)

Topic weight for each lyric - matrix shape:  (237, 5)


In [11]:
H = nmf.components_
print("Word weight for each topic - matrix shape: ", H.shape)

Word weight for each topic - matrix:  (5, 344)


In [12]:
for i, topic in enumerate(H):
    print(f'Topic {i+1}:')
    print(" ".join(vectorizer.get_feature_names_out()[i] for i in topic.argsort() [:-21:-1]))

Topic 1:
said die one life want high time make inside back baby see two never live come tell need take good
Topic 2:
like top rock em ya back dot boy rap money jay yo new coast big dawg still pop west let
Topic 3:
feel like nobody god pussy everything way life stop kill something feeling talkin heart real always damn work em young
Topic 4:
love people wanna give keep time away real still us black run way need one feeling would night another money
Topic 5:
know mean day wanna really let kendrick city girl new say told business hoes compton used real bout talk today


In [13]:
topic_labels = ['life and death', 'hip-hop', 'sentiment', 'love', 'himself']
topic_values = W
topic_values.shape

(237, 5)

In [14]:
topic_data_frame = pd.DataFrame(topic_values, columns = topic_labels)
topic_data_frame

,life and death,hip-hop,sentiment,love,himself
0,0.161416,0.031569,0.005612,0.022458,0.107169
1,0.104418,0.115086,0.000000,0.000000,0.033161
2,0.142371,0.061826,0.084336,0.043765,0.074203
3,0.038661,0.000000,0.027829,0.113632,0.177251
4,0.095714,0.094015,0.000000,0.041016,0.115280
...,...,...,...,...,...
232,0.021584,0.045298,0.000000,0.000313,0.054635
233,0.049321,0.217916,0.000000,0.013450,0.019778
234,0.166474,0.082181,0.002994,0.037250,0.120159
235,0.135927,0.071965,0.000000,0.082603,0.000000


In [16]:
data = data.join(topic_data_frame)
data.head()

,Track Title,Release Title,Release Subtitle,Track #,Track Length (mm:ss),Featuring,Release Type,Release Type #,Release Date,# of Tracks,...,Range Rover,RAV4,Silverado,Toyota,Van,life and death,hip-hop,sentiment,love,himself
0,Fuck Your Ethnicity,Section.80,NaN,1,3:44,Colin Munroe,Studio,1.0,2 Jul 11,16.0,...,NaN,NaN,NaN,NaN,NaN,0.161416,0.031569,0.005612,0.022458,0.107169
1,Hol' Up,Section.80,NaN,2,2:53,NaN,Studio,1.0,2 Jul 11,16.0,...,NaN,1.0,NaN,NaN,NaN,0.104418,0.115086,0.000000,0.000000,0.033161
2,A.D.H.D.,Section.80,NaN,3,3:35,NaN,Studio,1.0,2 Jul 11,16.0,...,NaN,NaN,NaN,NaN,NaN,0.142371,0.061826,0.084336,0.043765,0.074203
3,No Make-Up (Her Vice),Section.80,NaN,4,3:55,NaN,Studio,1.0,2 Jul 11,16.0,...,NaN,NaN,NaN,NaN,NaN,0.038661,0.000000,0.027829,0.113632,0.177251
4,Tammy's Song (Her Evils),Section.80,NaN,5,2:41,NaN,Studio,1.0,2 Jul 11,16.0,...,NaN,NaN,NaN,NaN,NaN,0.095714,0.094015,0.000000,0.041016,0.115280


In [ ]:
for i in topic_labels:
    data.loc[data[i] >= 0.1, i] = 1